# LMSys Prediction

In [1]:
class CFG:
    predict_strategies =  ["random", "voting", "modeling"]
    predict_strategy = predict_strategies[0]

In [2]:
import pandas as pd
import numpy as np
import json
import tensorflow as tf

2024-05-06 19:11:37.508596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 19:11:37.508740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 19:11:37.697485: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [4]:
test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")
test.head()

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [5]:
train.iloc[0].prompt

'["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]'

In [6]:
train.iloc[0].response_a

'["The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\\n\\nHere are some arguments in favor of and against such policies:\\n\\n**Arguments in favor:**\\n\\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\\n\\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\\n\\n3. **Equality of Opportunity:** Setting targets for female representation in management can help ensure that women have equal opportunities to advance in their careers.\\n\\n4. **R

In [7]:
train.iloc[0].response_b

'["As an AI, I don\'t have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better decision-making. Critics may argue that positions should be based purely on merit, and that quotas could potentially lead to tokenism or unqualified candidates being promoted. Morality can be subjective and differs from person to person based on their personal beliefs, cultural background, and values. It\'s a topic with valid arguments on both sides, and it\'s up to each individual or organization to decide what they believe is right.","As an AI, I don\'t eat, so I don\'t have personal preferences. But, in the world of pizza, the pineapple topping is a hotly debated topic! Some people love the sweet and savory mix it brings to a pizza, a flavor combination made famous by the Hawaiian pizza. Ot

In [8]:
train.winner_model_a.unique()

array([1, 0])

In [9]:
train.winner_model_b.unique()

array([0, 1])

In [10]:
train.winner_tie.unique()

array([0, 1])

In [11]:
import json
def prompt_length(prompt):
    return len(json.loads(prompt))
train["prompt_length"] = train["prompt"].apply(prompt_length)
train["prompt_length"].value_counts()

prompt_length
1     49938
2      4673
3      1485
4       607
5       311
6       167
7       103
8        60
9        44
10       29
11       16
12       14
13        5
19        4
15        4
14        3
17        3
16        2
20        2
36        1
18        1
21        1
35        1
22        1
29        1
32        1
Name: count, dtype: int64

In [12]:
submission = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/sample_submission.csv")
submission.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.333333,0.333333,0.333333
1,211333,0.333333,0.333333,0.333333
2,1233961,0.333333,0.333333,0.333333


In [13]:
if CFG.predict_strategy == CFG.predict_strategies[0]:
    result = np.random.rand(len(submission), 3)
    model = tf.keras.Sequential([
        tf.keras.Input((3,)),
        tf.keras.layers.Activation("softmax")
    ])
    result = model(result)
    submission[["winner_model_a", "winner_model_b", "winner_tie"]] = result
submission.to_csv("submission.csv", index=False)
submission.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.254705,0.444120,0.301175
1,211333,0.267492,0.388500,0.344009
2,1233961,0.244416,0.400163,0.355421
